# NLP - CEIA - 2024

## Entrega 2


### Autor: Julio Donadello
### Email: juliodonadello@gmail.com

---

# Custom embedddings con Gensim

## Ejercicio

- Tomar un ejemplo de los bots utilizados (uno de los dos) y construir el propio.
- Sacar conclusiones de los resultados.

__IMPORTANTE__: Recuerde para la entrega del ejercicio debe quedar registrado en el colab las preguntas y las respuestas del BOT para que podamos evaluar el desempeño final.

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

# Dataset: FIFA World Cup 2022 Tweets

In [1]:
ls

fifa_world_cup_2022_tweets.csv.zip  sample_data/


In [73]:
import zipfile

# variables si se sube el archivo al colab
zip_file = 'fifa_world_cup_2022_tweets.csv.zip'
csv_file = 'fifa_world_cup_2022_tweets.csv'

# variables ejecutando notebook local
#zip_file = './data/fifa_world_cup_2022_tweets.csv.zip'

with zipfile.ZipFile(zip_file, 'r') as z:
    with z.open(csv_file) as f:
        df = pd.read_csv(f)


df = df[['Tweet']]
df["Tweet"] = df["Tweet"].astype(str)
df.head()

,Tweet
0,What are we drinking today @TucanTribe \n@MadBears_ \n@lkinc_algo \n@al_goanna \n\n#WorldCup2022 https://t.co/Oga3TzvG5h
1,Amazing @CanadaSoccerEN #WorldCup2022 launch video. Shows how much the face of Canada and our men’s national team have changed since our last World Cup entry in 1986. Can’t wait to see these boys in action!\n\nThis is Canada: FIFA World Cup Opening Video https://t.co/7g73vvwtg8
2,Worth reading while watching #WorldCup2022 https://t.co/1SQrNa2dYU
3,Golden Maknae shinning bright\n\nhttps://t.co/4AyZbzGTX4\n#JeonJungkook #Jungkook #전정국 #정국 #JK #GoldenMaknae #bunny #Kookie #Jungshook #BTS #방탄소년단 #WorldCup2022 #FIFAKOOK \n@BTS_twt
4,"If the BBC cares so much about human rights, homosexual rights, and women rights then why not say these before the opening ceremony?? Why are they saying these during the opening ceremony?? Why did the BBC censor the #WorldCup2022 opening ceremony?? https://t.co/f72P03ZN2k"


In [74]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 22524


In [75]:
pd.set_option('display.max_colwidth', None)
print(df[:1])

                                                                                                                      Tweet
0  What are we drinking today @TucanTribe \n@MadBears_ \n@lkinc_algo \n@al_goanna \n\n#WorldCup2022 https://t.co/Oga3TzvG5h


## 1 - Preprocesamiento

### Eliminacion de urls

In [76]:
import re

def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

In [77]:
df["Tweet2"] = df["Tweet"].apply(remove_urls)
df.head()

,Tweet,Tweet2
0,What are we drinking today @TucanTribe \n@MadBears_ \n@lkinc_algo \n@al_goanna \n\n#WorldCup2022 https://t.co/Oga3TzvG5h,What are we drinking today @TucanTribe \n@MadBears_ \n@lkinc_algo \n@al_goanna \n\n#WorldCup2022
1,Amazing @CanadaSoccerEN #WorldCup2022 launch video. Shows how much the face of Canada and our men’s national team have changed since our last World Cup entry in 1986. Can’t wait to see these boys in action!\n\nThis is Canada: FIFA World Cup Opening Video https://t.co/7g73vvwtg8,Amazing @CanadaSoccerEN #WorldCup2022 launch video. Shows how much the face of Canada and our men’s national team have changed since our last World Cup entry in 1986. Can’t wait to see these boys in action!\n\nThis is Canada: FIFA World Cup Opening Video
2,Worth reading while watching #WorldCup2022 https://t.co/1SQrNa2dYU,Worth reading while watching #WorldCup2022
3,Golden Maknae shinning bright\n\nhttps://t.co/4AyZbzGTX4\n#JeonJungkook #Jungkook #전정국 #정국 #JK #GoldenMaknae #bunny #Kookie #Jungshook #BTS #방탄소년단 #WorldCup2022 #FIFAKOOK \n@BTS_twt,Golden Maknae shinning bright\n\n\n#JeonJungkook #Jungkook #전정국 #정국 #JK #GoldenMaknae #bunny #Kookie #Jungshook #BTS #방탄소년단 #WorldCup2022 #FIFAKOOK \n@BTS_twt
4,"If the BBC cares so much about human rights, homosexual rights, and women rights then why not say these before the opening ceremony?? Why are they saying these during the opening ceremony?? Why did the BBC censor the #WorldCup2022 opening ceremony?? https://t.co/f72P03ZN2k","If the BBC cares so much about human rights, homosexual rights, and women rights then why not say these before the opening ceremony?? Why are they saying these during the opening ceremony?? Why did the BBC censor the #WorldCup2022 opening ceremony??"


## Lowercase

In [82]:
df["Tweet2"] = df["Tweet2"].str.lower()
df.head()

,Tweet,Tweet2
0,What are we drinking today @TucanTribe \n@MadBears_ \n@lkinc_algo \n@al_goanna \n\n#WorldCup2022 https://t.co/Oga3TzvG5h,what drinking today @tucantribe @madbears_ @lkinc_algo @al_goanna #worldcup2022
1,Amazing @CanadaSoccerEN #WorldCup2022 launch video. Shows how much the face of Canada and our men’s national team have changed since our last World Cup entry in 1986. Can’t wait to see these boys in action!\n\nThis is Canada: FIFA World Cup Opening Video https://t.co/7g73vvwtg8,amazing @canadasocceren #worldcup2022 launch video. shows much face canada men’s national team changed since last world cup entry 1986. can’t wait see boys action! this canada: fifa world cup opening video
2,Worth reading while watching #WorldCup2022 https://t.co/1SQrNa2dYU,worth reading watching #worldcup2022
3,Golden Maknae shinning bright\n\nhttps://t.co/4AyZbzGTX4\n#JeonJungkook #Jungkook #전정국 #정국 #JK #GoldenMaknae #bunny #Kookie #Jungshook #BTS #방탄소년단 #WorldCup2022 #FIFAKOOK \n@BTS_twt,golden maknae shinning bright #jeonjungkook #jungkook #전정국 #정국 #jk #goldenmaknae #bunny #kookie #jungshook #bts #방탄소년단 #worldcup2022 #fifakook @bts_twt
4,"If the BBC cares so much about human rights, homosexual rights, and women rights then why not say these before the opening ceremony?? Why are they saying these during the opening ceremony?? Why did the BBC censor the #WorldCup2022 opening ceremony?? https://t.co/f72P03ZN2k","if bbc cares much human rights, homosexual rights, women rights say opening ceremony?? why saying opening ceremony?? why bbc censor #worldcup2022 opening ceremony??"


## Eliminacion de stop words

In [92]:
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
stopwords[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [84]:
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stopwords])

df["Tweet2"] = df["Tweet2"].apply(lambda text: remove_stopwords(text))
df.head()

,Tweet,Tweet2
0,What are we drinking today @TucanTribe \n@MadBears_ \n@lkinc_algo \n@al_goanna \n\n#WorldCup2022 https://t.co/Oga3TzvG5h,drinking today @tucantribe @madbears_ @lkinc_algo @al_goanna #worldcup2022
1,Amazing @CanadaSoccerEN #WorldCup2022 launch video. Shows how much the face of Canada and our men’s national team have changed since our last World Cup entry in 1986. Can’t wait to see these boys in action!\n\nThis is Canada: FIFA World Cup Opening Video https://t.co/7g73vvwtg8,amazing @canadasocceren #worldcup2022 launch video. shows much face canada men’s national team changed since last world cup entry 1986. can’t wait see boys action! canada: fifa world cup opening video
2,Worth reading while watching #WorldCup2022 https://t.co/1SQrNa2dYU,worth reading watching #worldcup2022
3,Golden Maknae shinning bright\n\nhttps://t.co/4AyZbzGTX4\n#JeonJungkook #Jungkook #전정국 #정국 #JK #GoldenMaknae #bunny #Kookie #Jungshook #BTS #방탄소년단 #WorldCup2022 #FIFAKOOK \n@BTS_twt,golden maknae shinning bright #jeonjungkook #jungkook #전정국 #정국 #jk #goldenmaknae #bunny #kookie #jungshook #bts #방탄소년단 #worldcup2022 #fifakook @bts_twt
4,"If the BBC cares so much about human rights, homosexual rights, and women rights then why not say these before the opening ceremony?? Why are they saying these during the opening ceremony?? Why did the BBC censor the #WorldCup2022 opening ceremony?? https://t.co/f72P03ZN2k","bbc cares much human rights, homosexual rights, women rights say opening ceremony?? saying opening ceremony?? bbc censor #worldcup2022 opening ceremony??"


## Eliminacion top 10 palabras mas frecuentes

In [79]:
from collections import Counter

In [85]:
cnt = Counter()
for text in df["Tweet2"].values:
    for word in text.split():
        cnt[word] += 1

cnt.most_common(10)

[('#worldcup2022', 21649),
 ('world', 5860),
 ('cup', 4536),
 ('qatar', 3598),
 ('#fifaworldcup', 3273),
 ('#worldcup', 3106),
 ('#qatar2022', 2762),
 ('opening', 2121),
 ('#qatarworldcup2022', 2062),
 ('2022', 1690)]

In [86]:
Freq = set([w for (w, wc) in cnt.most_common(10)])
def remove_freqwords(text):
    return " ".join([word for word in str(text).split() if word not in Freq])

df["Tweet2"] = df["Tweet2"].apply(lambda text: remove_freqwords(text))
df.head()

,Tweet,Tweet2
0,What are we drinking today @TucanTribe \n@MadBears_ \n@lkinc_algo \n@al_goanna \n\n#WorldCup2022 https://t.co/Oga3TzvG5h,drinking today @tucantribe @madbears_ @lkinc_algo @al_goanna
1,Amazing @CanadaSoccerEN #WorldCup2022 launch video. Shows how much the face of Canada and our men’s national team have changed since our last World Cup entry in 1986. Can’t wait to see these boys in action!\n\nThis is Canada: FIFA World Cup Opening Video https://t.co/7g73vvwtg8,amazing @canadasocceren launch video. shows much face canada men’s national team changed since last entry 1986. can’t wait see boys action! canada: fifa video
2,Worth reading while watching #WorldCup2022 https://t.co/1SQrNa2dYU,worth reading watching
3,Golden Maknae shinning bright\n\nhttps://t.co/4AyZbzGTX4\n#JeonJungkook #Jungkook #전정국 #정국 #JK #GoldenMaknae #bunny #Kookie #Jungshook #BTS #방탄소년단 #WorldCup2022 #FIFAKOOK \n@BTS_twt,golden maknae shinning bright #jeonjungkook #jungkook #전정국 #정국 #jk #goldenmaknae #bunny #kookie #jungshook #bts #방탄소년단 #fifakook @bts_twt
4,"If the BBC cares so much about human rights, homosexual rights, and women rights then why not say these before the opening ceremony?? Why are they saying these during the opening ceremony?? Why did the BBC censor the #WorldCup2022 opening ceremony?? https://t.co/f72P03ZN2k","bbc cares much human rights, homosexual rights, women rights say ceremony?? saying ceremony?? bbc censor ceremony??"


## Steeming

In [87]:
from nltk.stem.porter import PorterStemmer

In [88]:
Pstemmer = PorterStemmer()
def stem_words(text):
    return " ".join([Pstemmer.stem(word) for word in text.split()])

df["Tweet2"] = df["Tweet2"].apply(lambda text: stem_words(text))
df.head()

,Tweet,Tweet2
0,What are we drinking today @TucanTribe \n@MadBears_ \n@lkinc_algo \n@al_goanna \n\n#WorldCup2022 https://t.co/Oga3TzvG5h,drink today @tucantrib @madbears_ @lkinc_algo @al_goanna
1,Amazing @CanadaSoccerEN #WorldCup2022 launch video. Shows how much the face of Canada and our men’s national team have changed since our last World Cup entry in 1986. Can’t wait to see these boys in action!\n\nThis is Canada: FIFA World Cup Opening Video https://t.co/7g73vvwtg8,amaz @canadasocceren launch video. show much face canada men’ nation team chang sinc last entri 1986. can’t wait see boy action! canada: fifa video
2,Worth reading while watching #WorldCup2022 https://t.co/1SQrNa2dYU,worth read watch
3,Golden Maknae shinning bright\n\nhttps://t.co/4AyZbzGTX4\n#JeonJungkook #Jungkook #전정국 #정국 #JK #GoldenMaknae #bunny #Kookie #Jungshook #BTS #방탄소년단 #WorldCup2022 #FIFAKOOK \n@BTS_twt,golden makna shin bright #jeonjungkook #jungkook #전정국 #정국 #jk #goldenmakna #bunni #kooki #jungshook #bt #방탄소년단 #fifakook @bts_twt
4,"If the BBC cares so much about human rights, homosexual rights, and women rights then why not say these before the opening ceremony?? Why are they saying these during the opening ceremony?? Why did the BBC censor the #WorldCup2022 opening ceremony?? https://t.co/f72P03ZN2k","bbc care much human rights, homosexu rights, women right say ceremony?? say ceremony?? bbc censor ceremony??"


## Puntuacion

In [89]:
import string

In [98]:
def remove_punctuation(text):
    return " ".join([word for word in text.split() if word not in string.punctuation])

df["Tweet2"] = df["Tweet2"].apply(lambda text: remove_punctuation(text))

df.head(10)

,Tweet,Tweet2
0,What are we drinking today @TucanTribe \n@MadBears_ \n@lkinc_algo \n@al_goanna \n\n#WorldCup2022 https://t.co/Oga3TzvG5h,drink today @tucantrib @madbears_ @lkinc_algo @al_goanna
1,Amazing @CanadaSoccerEN #WorldCup2022 launch video. Shows how much the face of Canada and our men’s national team have changed since our last World Cup entry in 1986. Can’t wait to see these boys in action!\n\nThis is Canada: FIFA World Cup Opening Video https://t.co/7g73vvwtg8,amaz @canadasocceren launch video. show much face canada men’ nation team chang sinc last entri 1986. can’t wait see boy action! canada: fifa video
2,Worth reading while watching #WorldCup2022 https://t.co/1SQrNa2dYU,worth read watch
3,Golden Maknae shinning bright\n\nhttps://t.co/4AyZbzGTX4\n#JeonJungkook #Jungkook #전정국 #정국 #JK #GoldenMaknae #bunny #Kookie #Jungshook #BTS #방탄소년단 #WorldCup2022 #FIFAKOOK \n@BTS_twt,golden makna shin bright #jeonjungkook #jungkook #전정국 #정국 #jk #goldenmakna #bunni #kooki #jungshook #bt #방탄소년단 #fifakook @bts_twt
4,"If the BBC cares so much about human rights, homosexual rights, and women rights then why not say these before the opening ceremony?? Why are they saying these during the opening ceremony?? Why did the BBC censor the #WorldCup2022 opening ceremony?? https://t.co/f72P03ZN2k","bbc care much human rights, homosexu rights, women right say ceremony?? say ceremony?? bbc censor ceremony??"
5,"And like, will the mexican fans be able to scream ""PUTO"" now? Or is that too homophobic for qatar? @FIFAWorldCup #WorldCup2022","like, mexican fan abl scream ""puto"" now? homophob qatar? @fifaworldcup"
6,Look like a only me and the Jamaican football team naw follow up worldcup #WorldCup2022,look like jamaican footbal team naw follow worldcup
7,Really? Football on a Monday morning at 9 and 12 and then 3? I need to pinch myself. Is it really happening? \n#WorldCup2022,really? footbal monday morn 9 12 3? need pinch myself. realli happening?
8,"As the World Cup starts in Qatar, it’s Black Awareness Day in Brazil✊🏽. Despite the atrocities linked to this year’s host and to Fifa, soccer is so fundamental for lower classes, mostly Black and Brown, as Vini Jr identifies #WorldCup2022 #diadaconsciencianegra #CopaDoMundo2022 https://t.co/8mI9UqNWX7","start qatar, it’ black awar day brazil✊🏽. despit atroc link year’ host fifa, soccer fundament lower classes, mostli black brown, vini jr identifi #diadaconsciencianegra #copadomundo2022"
9,#WorldCup2022 @ITVSport &amp; @LFSYSTEMMUSIC go together so well ‘Hungry For Your Love’🫶 ⚽️👌,@itvsport &amp; @lfsystemmus go togeth well ‘hungri love’🫶 ⚽️👌


In [96]:
"can’t" in stopwords

False

In [105]:
df_final = df[["Tweet2"]]
df_final.head()

,Tweet2
0,drink today @tucantrib @madbears_ @lkinc_algo @al_goanna
1,amaz @canadasocceren launch video. show much face canada men’ nation team chang sinc last entri 1986. can’t wait see boy action! canada: fifa video
2,worth read watch
3,golden makna shin bright #jeonjungkook #jungkook #전정국 #정국 #jk #goldenmakna #bunni #kooki #jungshook #bt #방탄소년단 #fifakook @bts_twt
4,"bbc care much human rights, homosexu rights, women right say ceremony?? say ceremony?? bbc censor ceremony??"


## Segmentacion del texto

In [106]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df_final[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

<ipython-input-106-db3da403184b>:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sentence_tokens.append(text_to_word_sequence(row[0]))


In [110]:
sentence_tokens[:2]

[['drink', 'today', 'tucantrib', 'madbears', 'lkinc', 'algo', 'al', 'goanna'],
 ['amaz',
  'canadasocceren',
  'launch',
  'video',
  'show',
  'much',
  'face',
  'canada',
  'men’',
  'nation',
  'team',
  'chang',
  'sinc',
  'last',
  'entri',
  '1986',
  'can’t',
  'wait',
  'see',
  'boy',
  'action',
  'canada',
  'fifa',
  'video']]

## 2 - Crear los vectores (word2vec)

In [111]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [122]:
vector_size = 50
for row in df_final['Tweet2']:
  if len(row) > vector_size:
    vector_size = len(row)
print(vector_size)

361


In [123]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size= vector_size,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [124]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [125]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 22524


In [126]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 4886


## 3 - Entrenar embeddings

In [127]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=50,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 2283013.0
Loss after epoch 1: 1371905.5
Loss after epoch 2: 1283205.0
Loss after epoch 3: 1218217.5
Loss after epoch 4: 1184136.0
Loss after epoch 5: 1149618.0
Loss after epoch 6: 1080356.0
Loss after epoch 7: 1056824.0
Loss after epoch 8: 1032361.0
Loss after epoch 9: 1009853.0
Loss after epoch 10: 987282.0
Loss after epoch 11: 966840.0
Loss after epoch 12: 952957.0
Loss after epoch 13: 934313.0
Loss after epoch 14: 871915.0
Loss after epoch 15: 838870.0
Loss after epoch 16: 826212.0
Loss after epoch 17: 808370.0
Loss after epoch 18: 792966.0
Loss after epoch 19: 778676.0
Loss after epoch 20: 769068.0
Loss after epoch 21: 756774.0
Loss after epoch 22: 745986.0
Loss after epoch 23: 735684.0
Loss after epoch 24: 725672.0
Loss after epoch 25: 716140.0
Loss after epoch 26: 709240.0
Loss after epoch 27: 702928.0
Loss after epoch 28: 694544.0
Loss after epoch 29: 688020.0
Loss after epoch 30: 680008.0
Loss after epoch 31: 675920.0
Loss after epoch 32: 668850.0
Loss after

(9338678, 11770150)

In [128]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["goal"], topn=10)

[('visit', 0.009057419374585152),
 ('chat', 0.0027378019876778126),
 ('globally', -0.004894433543086052),
 ('fellow', -0.011399630457162857),
 ('eu', -0.014946666546165943),
 ('sundayvib', -0.01807096041738987),
 ('abu', -0.02313833497464657),
 ('🇲🇽', -0.02515852451324463),
 ('refer', -0.025229420512914658),
 ('colour', -0.025758692994713783)]

In [131]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["goal"], topn=10)

[('hattrick', 0.4571785032749176),
 ('disallowed', 0.42226043343544006),
 ('perfectli', 0.4140697717666626),
 ('disallow', 0.3940123915672302),
 ('bastard', 0.38618203997612),
 ('worldcupopeningmatch', 0.38379132747650146),
 ('ener', 0.37355250120162964),
 ('gol', 0.3711113929748535),
 ("16'", 0.370531290769577),
 ('ennervalencia𓃵', 0.3696558177471161)]

In [143]:
w2v_model.wv.index_to_key[0:20]

['ecuador',
 'qatar',
 'fifa',
 'goal',
 'footbal',
 'go',
 'game',
 'team',
 'first',
 'ceremoni',
 'watch',
 'start',
 'get',
 'match',
 'cup',
 'win',
 'today',
 'worldcupqatar2022',
 'var',
 'worldcup2022']

In [146]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["messi"], topn=5)

[('lionel', 0.55973881483078),
 ('legends', 0.4650469124317169),
 ('assists', 0.45216020941734314),
 ('ronaldo', 0.4492332935333252),
 ('lionelmessi', 0.44461551308631897)]

In [151]:
w2v_model.wv.most_similar(positive=["enner"], topn=7)

[('valencia', 0.5750347971916199),
 ("16'", 0.5200327634811401),
 ('ener', 0.5089389681816101),
 ("15'", 0.48304668068885803),
 ('fenerbahc', 0.45728328824043274),
 ('valenci̇a', 0.45321378111839294),
 ('ennervalencia𓃵', 0.44991645216941833)]

## 4 - Visualizar agrupación de vectores

In [165]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [166]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

Se pueden observar algunas similaridades interesantes:

- emoji de risa con 'var'
- 'money' con 'hope'
- 'great' con 'nation'
- 'brasil', 'argentina' y 'final'
- 'ceremoni' con emoji de tv

In [167]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

# 5 - Conclusiones

Se lograron resultados interesantes y coherentes. Se logro usar un modelo de NLP como herramienta de analisis de tweets con busqueda 'automatica' de insights.
Sirve para analizar el pensamiento generalizado de un grupo de personas.

Por otro lado, en lo que refiere al modelo, se puede seguir mejorando mediante un mayor preprocesamiento y mediante distintos ciclos de entrenamiento utilizando nuevos valores para los parametros de Word2Vec (especialmente "negative" y "vector_size") y para el hiperparametro de epochs.